<a href="https://colab.research.google.com/github/nah90/CS777_PySparkTFIDF/blob/main/MET_CS_777_Assignment_2_Horak.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###BU MET CS 777 ASSIGNMENT 2###

In [ ]:
!pip install --ignore-installed -q pyspark==3.2.1

In [ ]:
import sys
import re
import numpy as np
import nltk

from numpy import dot
from numpy.linalg import norm
from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

sc = SparkContext.getOrCreate()
spark = SparkSession.builder.getOrCreate()
sqlContext = SQLContext(sc)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [ ]:
# Set the file paths on your local machine
# Change this line later on your python script when you want to run this on the CLOUD (GC or AWS)


wikiPagesFile="WikipediaPagesOneDocPerLine1000LinesSmall.txt.bz2"
wikiCategoryFile="wiki-categorylinks-small.csv.bz2"
#wikiPagesFile="gs://met-cs-777-data/WikipediaPagesOneDocPerLine1000LinesSmall.txt.bz2"
#wikiCategoryFile="gs://met-cs-777-data/wiki-categorylinks-small.csv.bz2"

In [ ]:
# Read two files into RDDs

wikiCategoryLinks=sc.textFile(wikiCategoryFile)

wikiCats=wikiCategoryLinks.map(lambda x: x.split(",")).map(lambda x: (x[0].replace('"', ''), x[1].replace('"', '') ))

# Now the wikipages
wikiPages = sc.textFile(wikiPagesFile)

wikiCategoryLinks.take(2)

['"434042","1987_debut_albums"', '"434042","Albums_produced_by_Mike_Varney"']

In [ ]:
wikiCats.take(1)

[('434042', '1987_debut_albums')]

In [ ]:
df = spark.read.csv(wikiPagesFile)

# Uncomment this line if you want to take look inside the file. 
#df.take(1)

In [ ]:
# Assumption: Each document is stored in one line of the text file
# We need this count later ... 
numberOfDocs = wikiPages.count()

print(numberOfDocs)
# Each entry in validLines will be a line from the text file
validLines = wikiPages.filter(lambda x : 'id' in x and 'url=' in x)

# Now, we transform it into a set of (docID, text) pairs
keyAndText = validLines.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('">') + 2:][:-6])) 

1000


In [ ]:
def buildArray(listOfIndices):
    
    returnVal = np.zeros(20000)
    
    for index in listOfIndices:
        returnVal[index] = returnVal[index] + 1
    
    mysum = np.sum(returnVal)
    
    returnVal = np.divide(returnVal, mysum)
    
    return returnVal


def build_zero_one_array (listOfIndices):
    
    returnVal = np.zeros (20000)
    
    for index in listOfIndices:
        if returnVal[index] == 0: returnVal[index] = 1
    
    return returnVal


def stringVector(x):
    returnVal = str(x[0])
    for j in x[1]:
        returnVal += ',' + str(j)
    return returnVal



def cousinSim (x,y):
	normA = np.linalg.norm(x)
	normB = np.linalg.norm(y)
	return np.dot(x,y)/(normA*normB)

In [ ]:
#TASK 1
# Now, we transform it into a set of (docID, text) pairs
keyAndText = validLines.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('">') + 2:][:-6]))

# Now, we split the text in each (docID, text) pair into a list of words
# After this step, we have a data set with
# (docID, ["word1", "word2", "word3", ...])
# We use a regular expression here to make
# sure that the program does not break down on some of the documents

regex = re.compile('[^a-zA-Z]')

# remove all non letter characters
keyAndListOfWords = keyAndText.map(lambda x : (str(x[0]), regex.sub(' ', x[1]).lower().split()))
# better solution here is to use NLTK tokenizer

# Now get the top 20,000 words... first change (docID, ["word1", "word2", "word3", ...])
# to ("word1", 1) ("word2", 1)...

allWords = keyAndListOfWords.flatMap(lambda x: [(a,1) for a in x[1]])



####TO REMOVE STOPWORDS

#nltk.download('stopwords')
#stopwords = list(stopwords.words('english'))
#allWords = allWords.filter(lambda x: x[0] not in stopwords)

#stemming
#stemmer = PorterStemmer()
#allWords = allWords.filter(lambda x: stemmer.stem(x[0]))

#print(allWords.take(100))

####




# Now, count all of the words, giving us ("word1", 1433), ("word2", 3423423), etc.
allCounts = allWords.reduceByKey(lambda x, y: x+y)

#print(allCounts.count())

# Get the top 20,000 words in a local array in a sorted format based on frequency
# If you want to run it on your laptio, it may a longer time for top 20k words. 
topWords = allCounts.top(20000, lambda x: x[1])

print("Top Words in Corpus:", allCounts.top(20, key=lambda x: x[1]))

# We'll create a RDD that has a set of (word, dictNum) pairs
# start by creating an RDD that has the number 0 through 20000
# 20000 is the number of words that will be in our dictionary
topWordsK = sc.parallelize(range(20000))

# Now, we transform (0), (1), (2), ... to ("MostCommonWord", 1)
# ("NextMostCommon", 2), ...
# the number will be the spot in the dictionary used to tell us
# where the word is located
dictionary = topWordsK.map (lambda x : (topWords[x][0], x))
dict_data = dictionary.collectAsMap()
sc.broadcast(dict_data)

print("Word Postions in our Feature Matrix. Last 20 words in 20k positions: ", dictionary.top(20, lambda x : x[1]))

Top Words in Corpus: [('the', 74530), ('of', 34512), ('and', 28479), ('in', 27758), ('to', 22583), ('a', 21212), ('was', 12160), ('as', 8811), ('for', 8773), ('on', 8435), ('is', 8392), ('s', 8291), ('by', 7483), ('with', 7108), ('that', 6935), ('he', 5552), ('from', 5216), ('his', 5041), ('at', 4810), ('it', 4442)]
Word Postions in our Feature Matrix. Last 20 words in 20k positions:  [('quebecor', 19999), ('poten', 19998), ('kasada', 19997), ('yadnya', 19996), ('drift', 19995), ('iata', 19994), ('satire', 19993), ('expreso', 19992), ('olimpico', 19991), ('auxiliaries', 19990), ('tenses', 19989), ('petherick', 19988), ('stowe', 19987), ('infimum', 19986), ('parramatta', 19985), ('rimpac', 19984), ('hyderabad', 19983), ('cubes', 19982), ('meats', 19981), ('chaat', 19980)]


In [ ]:
# Next, we get a RDD that has, for each (docID, ["word1", "word2", "word3", ...]),
# ("word1", docID), ("word2", docId), ...

allWordsWithDocID = keyAndListOfWords.flatMap(lambda x: ((j, x[0]) for j in x[1]))

# Now join and link them, to get a set of ("word1", (docID, dictionaryPos)) pairs
allDictionaryWords = allWordsWithDocID.map(lambda x: (x[0], (x[1], dict_data.get(x[0]))) if x[0] in dict_data.keys() else None).filter(lambda x: x!=None)
#allDictionaryWords = dictionary.join(allWordsWithDocID)

# Now, we drop the actual word itself to get a set of (docID, dictionaryPos) pairs
justDocAndPos = allDictionaryWords.map(lambda x: (x[1][0], x[1][1]))

# Now get a set of (docID, [dictionaryPos1, dictionaryPos2, dictionaryPos3...]) pairs
allDictionaryWordsInEachDoc = justDocAndPos.groupByKey()

# The following line this gets us a set of
# (docID,  [dictionaryPos1, dictionaryPos2, dictionaryPos3...]) pairs
# and converts the dictionary positions to a bag-of-words numpy array...
allDocsAsNumpyArrays = allDictionaryWordsInEachDoc.map(lambda x: (x[0], buildArray(x[1])))

print(allDocsAsNumpyArrays.take(3))

[('431949', array([0.07558791, 0.02900336, 0.02329227, ..., 0.        , 0.        ,
       0.        ])), ('431952', array([0.08850932, 0.02950311, 0.04658385, ..., 0.        , 0.        ,
       0.        ])), ('431959', array([0.08685662, 0.04136029, 0.03584559, ..., 0.        , 0.        ,
       0.        ]))]


In [ ]:
# Now, create a version of allDocsAsNumpyArrays where, in the array,
# every entry is either zero or one.
# A zero means that the word does not occur,
# and a one means that it does.

zeroOrOne = allDocsAsNumpyArrays.map(lambda x: ("", np.clip(np.multiply(x[1], 9e9), 0, 1) ) )

# Now, add up all of those arrays into a single array, where the
# i^th entry tells us how many
# individual documents the i^th word in the dictionary appeared in
dfArray = zeroOrOne.reduce(lambda x1, x2: ("", np.add(x1[1], x2[1])))[1]

# Create an array of 20,000 entries, each entry with the value numberOfDocs (number of docs)
multiplier = np.full(20000, numberOfDocs)

# Get the version of dfArray where the i^th entry is the inverse-document frequency for the
# i^th word in the corpus
idfArray = np.log(np.divide(multiplier, dfArray))
sc.broadcast(idfArray)

# Finally, convert all of the tf vectors in allDocsAsNumpyArrays to tf * idf vectors
allDocsAsNumpyArraysTFidf = allDocsAsNumpyArrays.map(lambda x: (x[0], np.multiply(x[1], idfArray)))

print(allDocsAsNumpyArraysTFidf.take(2))

[('431949', array([0.00638485, 0.00241835, 0.00327052, ..., 0.        , 0.        ,
       0.        ])), ('431952', array([0.00747631, 0.00246002, 0.00654094, ..., 0.        , 0.        ,
       0.        ]))]


In [ ]:
#TASK 2
docIDs = set(allDocsAsNumpyArraysTFidf.map(lambda x: x[0]).collect())
sc.broadcast(docIDs)

print(wikiCats.take(1))
print(docIDs)

[('434042', '1987_debut_albums')]
{'432283', '418323', '418434', '432131', '429298', '456158', '456245', '455271', '455191', '418442', '429220', '455907', '418436', '455347', '431959', '429267', '429258', '456146', '418524', '455987', '434027', '433584', '418516', '433888', '429291', '455844', '434065', '418479', '433481', '455212', '433630', '432346', '433950', '456302', '432318', '429144', '456074', '429348', '429234', '429280', '456091', '455870', '456243', '456343', '429137', '455198', '455199', '455035', '418356', '429227', '433941', '418638', '418544', '418599', '418444', '418611', '433653', '428956', '429155', '429172', '433955', '418664', '429326', '433616', '429203', '418592', '455798', '432145', '454988', '418373', '418450', '455741', '433821', '429266', '418605', '418540', '455096', '456145', '433664', '433798', '428991', '418582', '433739', '429323', '433475', '429253', '454907', '429306', '433856', '429211', '454875', '432000', '455317', '418512', '456028', '418580', '4293

In [ ]:
allDocsCatsRDD = wikiCats.map(lambda x: (x[0], x[1] if x[0] in docIDs else None)).filter(lambda x: x[1] != None)

print(allDocsCatsRDD.take(1))

[('434042', '1987_debut_albums')]


In [ ]:
# Finally, we have a function that returns the prediction for the label of a string, using a kNN algorithm
def getPrediction (textInput, k):
    # Create an RDD out of the textIput
    myDoc = sc.parallelize (('', textInput))

    # Flat map the text to (word, 1) pair for each word in the doc
    wordsInThatDoc = myDoc.flatMap (lambda x : ((j, 1) for j in regex.sub(' ', x).lower().split()))

    # This will give us a set of (word, (dictionaryPos, 1)) pairs
    allDictionaryWordsInThatDoc = wordsInThatDoc.map(lambda x: (x[0], (x[1], dict_data.get(x[0]))) if x[0] in dict_data.keys() else None).filter(lambda x: x!=None).map (lambda x: (x[1][0], x[1][1])).groupByKey()

    # Get tf array for the input string
    myArray = buildArray (allDictionaryWordsInThatDoc.top (1)[0][1])

    # Get the tf * idf array for the input string
    myArray = np.multiply (myArray, idfArray)

    # Get the distance from the input text string to all database documents, using cosine similarity (np.dot() )
    distances = allDocsAsNumpyArraysTFidf.map (lambda x : (x[0], cousinSim (x[1],myArray))) #Use this line to calculate distance without featuresRDD
    # get the top k distances
    topK = distances.top (k, lambda x : x[1])
    
    # and transform the top k distances into a set of (docID, 1) pairs
    docIDRepresented = sc.parallelize(topK).map (lambda x : (x[0], 1))

    # now, for each docID, get the count of the number of times this document ID appeared in the top k
    numTimes = docIDRepresented.reduceByKey(lambda x, y : x + y)
    numTimes = allDocsCatsRDD.join(numTimes).map(lambda x: x[1]) #Joining here is much easier/faster than joining featuresRDD with wikiCats


    # Return the top 1 of them.
    return numTimes.top(k, lambda x: x[1])

In [ ]:
print(getPrediction('Sport Basketball Volleyball Soccer', 10))

[('1991_establishments_in_Bulgaria', 1), ('All_articles_with_dead_external_links', 1), ('Articles_with_dead_external_links_from_September_2019', 1), ('Articles_with_permanently_dead_external_links', 1), ('Commons_category_link_is_on_Wikidata', 1), ('Coordinates_on_Wikidata', 1), ('Educational_institutions_established_in_1991', 1), ('Liberal_arts_colleges', 1), ('Official_website_different_in_Wikidata_and_Wikipedia', 1), ('Pages_using_infobox_university_with_the_affiliations_parameter', 1)]


In [ ]:
print(getPrediction('What is the capital city of Australia?', 10))

[('All_Wikipedia_articles_written_in_Australian_English', 1), ('All_set_index_articles', 1), ('Articles_with_short_description', 1), ('Royal_Australian_Navy_ship_names', 1), ('Set_indices_on_ships', 1), ('Use_Australian_English_from_April_2018', 1), ('Use_dmy_dates_from_April_2018', 1), ('All_Wikipedia_articles_written_in_Australian_English', 1), ('All_set_index_articles', 1), ('Articles_with_short_description', 1)]


In [ ]:
print(getPrediction('How many goals Vancouver score last year?', 10))

[('All_Wikipedia_articles_in_need_of_updating', 1), ('Articles_with_obsolete_information_from_October_2019', 1), ('Expert_systems', 1), ('Inference', 1), ('Pages_using_web_citations_with_no_URL', 1), ('1990s_Canadian_teen_drama_television_series', 1), ('1991_Canadian_television_series_debuts', 1), ('1994_Canadian_television_series_endings', 1), ('All_stub_articles', 1), ('CBC_Television_shows', 1)]


In [ ]:
# Congratulations, you have implemented a prediction system based on Wikipedia data. 
# You can use this system to generate automated Tags or Categories for any kind of text 
# that you put in your query.
# This data model can predict categories for any input text. 

In [ ]:
#TASK 3
docsCatsDF = sqlContext.createDataFrame(allDocsCatsRDD, ['docID', 'category'])
docsCatsDF = docsCatsDF.groupBy('category').agg({'docID': 'count'})
print(docsCatsDF.select('count(docID)').summary().show())
print(docsCatsDF.orderBy('count(docID)', ascending = False).limit(10).show())

+-------+------------------+
|summary|      count(docID)|
+-------+------------------+
|  count|              7153|
|   mean|1.9264644205228576|
| stddev|  8.43216550055147|
|    min|                 1|
|    25%|                 1|
|    50%|                 1|
|    75%|                 1|
|    max|               273|
+-------+------------------+

None
+--------------------+------------+
|            category|count(docID)|
+--------------------+------------+
|Articles_with_sho...|         273|
|All_articles_with...|         222|
|Wikipedia_article...|         212|
|Wikipedia_article...|         212|
|Wikipedia_article...|         196|
|Commons_category_...|         178|
|Wikipedia_article...|         157|
|Wikipedia_article...|         155|
|All_articles_need...|         154|
|Webarchive_templa...|         140|
+--------------------+------------+

None


In [ ]:
sc.stop()
spark.stop()